In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense
from keras.utils import np_utils
from keras import backend as K

from keras import callbacks
from multiprocessing import Pool 
from multiprocessing.dummy import Pool as ThreadPool 
import tensorflow as tf
import pdb
from matplotlib import pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [ ]:
input_unit_size = 28*28
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
def draw_digit(data, row, col, n):
    size = int(np.sqrt(data.shape[0]))
    plt.subplot(row, col, n)    
    plt.imshow(data.reshape(size, size))
    plt.gray()

In [ ]:
x_train = x_train.reshape(x_train.shape[0], input_unit_size)
x_train = x_train.astype('float32')
x_train /= 255

In [ ]:
input_unit_size

In [ ]:
weight_folder = "weight_history_better/"

In [ ]:
wich_steps = list(range(0, 800, 50))
epochs = 2
class WeightHistory(callbacks.Callback):
    def  __init__(self, exp):
        self.exp = exp
        self.epoch = 1
        self.max_epochs = epochs
        self.step = 0
        
    def on_epoch_end(self, epoch, logs={}): 
        self.step = 0
        self.epoch += 1
        return 

    def on_batch_end(self, batch, logs={}):
        
        if self.step in wich_steps:
            
            list_connections = self.model.layers[1].get_weights()[0].flatten()
            m = np.reshape(list_connections, (336, 336))
            filename_weights = "{}/weight_epoch={}_ensemble={}_step={}.txt".format(weight_folder, self.epoch, self.exp, self.step)
            np.savetxt(filename_weights, m)

            #filename_eigvals = "{}/eigvals_epoch={}_ensemble={}_step={}.txt".format(weight_folder, self.epoch, self.exp, self.step)
            #eigvals = np.linalg.eigvals(m )
            #np.savetxt(filename_eigvals, eigvals.view(float))
    
            get_layer_output = K.function([model.layers[0].input],
                                          [model.layers[1].output])
            hidden_outputs = get_layer_output([x_train[0:show_size**2]])[0]
            size = int(np.sqrt(hidden_outputs[1].shape[0]))
            
            filename_hidden = "{}/hidden_ensemble={}_epoch={}_step={}.txt".format(weight_folder, self.epoch, self.exp, self.step)
            np.savetxt(filename_hidden, hidden_outputs[1].reshape(size, size))
            
            
        self.step +=1
        return


def create_exp(i):
    inputs = Input(shape=(input_unit_size,))
    x = Dense(144, activation='relu')(inputs)
    outputs = Dense(input_unit_size)(x)
    model = Model(input=inputs, output=outputs)
    
    #model.compile(loss=custom_objective, optimizer='adadelta', )
    model.compile(loss='binary_crossentropy', optimizer='adadelta', )

    model.fit(x_train, x_train, epochs=2,  batch_size=256, callbacks=[WeightHistory(i)],verbose=0  )
    #return model




 #get_layer_output = K.function([model.layers[0].input],
#                              [model.layers[1].output])

#hidden_outputs = get_layer_output([x_train[0:show_size**2]])[0]



In [ ]:
pool = Pool()
pool.map(create_exp, range(0, 100))
pool.close() 
pool.join()

In [ ]:
# raw image

show_size = 10
total = 0
plt.figure(figsize=(20, 20))
for i in range(show_size):
    for j in range(show_size):    
        draw_digit(x_train[total], show_size, show_size, total+1)
        total+=1
plt.show()

In [ ]:
# hidden
get_layer_output = K.function([model.layers[0].input],
                              [model.layers[1].output])

hidden_outputs = get_layer_output([x_train[0:show_size**2]])[0]

total = 0
plt.figure(figsize=(20, 20))
for i in range(show_size):
    for j in range(show_size):    
        draw_digit(hidden_outputs[total], show_size, show_size, total+1)
        total+=1
plt.show()

In [ ]:
# reconstruct image
get_layer_output = K.function([model.layers[0].input],
                              [model.layers[2].output])

last_outputs = get_layer_output([x_train[0:show_size**2]])[0]

total = 0
plt.figure(figsize=(20, 20))
for i in range(show_size):
    for j in range(show_size):    
        draw_digit(last_outputs[total], show_size, show_size, total+1)
        total+=1
plt.show()